# Prepare driver

In [1]:
! wget -P /home/jovyan https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar

--2019-07-06 16:35:58--  https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.84.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.84.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘/home/jovyan/postgresql-42.2.5.jar’

postgresql-42.2.5.j 100%[===================>] 806.58K  5.17MB/s    in 0.2s    

2019-07-06 16:35:58 (5.17 MB/s) - ‘/home/jovyan/postgresql-42.2.5.jar’ saved [825943/825943]



# Start spark application

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/jovyan/postgresql-42.2.5.jar --jars /home/jovyan/postgresql-42.2.5.jar pyspark-shell'

import pyspark

spark = pyspark.sql.SparkSession.builder \
        .master("local[1]") \
        .appName("snapshot") \
        .getOrCreate()

print("Application started")

Application started


# Warm-up spark

In [3]:
spark.sparkContext.range(1000).sum()
print("Spark application is ready for work")

Spark application is ready for work


# Read some PostgreSQL Data

In [36]:
customers = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='customers'
    ).load()

products = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='products'
    ).load()

orders = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='orders'
    ).load()

customers.registerTempTable("customers")
products.registerTempTable("products")
orders.registerTempTable("orders")

print("Customers table")
customers.show(10)
print("Orders table")
orders.show(10)
print("Products table")
products.show(10)


Customers table
+----+-----------+---------+--------------------+
|  id| first_name|last_name|               email|
+----+-----------+---------+--------------------+
|1099|     Dennis|     Cruz|zwolfe@berry-powe...|
|2156|       Lori|   Turner| dthomas@andrade.biz|
|2157|    Matthew|  Gregory|christianhammond@...|
|1572|  Christina|     West|reyesmichelle@gar...|
|2158|     Austin|  Hammond|gomezjames@martin...|
|2159|     Sharon|     Hill|    bmoreno@cruz.com|
|1573|Christopher|   Davila|lewiskristin@gmai...|
|1262|      Peggy|   Hughes|watsonmelinda@tur...|
|1003|     Edward|   Walker|daniellecarter@ho...|
|1264|    Kristin|    Glass|hardykiara@davis-...|
+----+-----------+---------+--------------------+
only showing top 10 rows

Orders table
+-----+----------+---------+--------+----------+
|   id|order_date|purchaser|quantity|product_id|
+-----+----------+---------+--------+----------+
|10001|2016-01-16|     1001|       1|       102|
|10002|2016-01-17|     1002|       2|       105|


# Read and join the data

In [40]:
# please write your query here
query = """ 
        select
            c.id, c.first_name, c.last_name, 
            coalesce(sum(p.weight),0) as total_weight, 
            current_timestamp as load_dttm
        from customers c 
            left join orders o on c.id=o.purchaser 
            left join products p on o.product_id=p.id 
        where c.id<=1005 
        group by c.id, c.first_name, c.last_name
        """
result = spark.sql(query)
result.write.format("parquet").mode("overwrite").save("/home/jovyan/weight_report")

In [41]:
spark.read \
    .format("parquet").load("/home/jovyan/weight_report").show()

+----+----------+---------+------------+--------------------+
|  id|first_name|last_name|total_weight|           load_dttm|
+----+----------+---------+------------+--------------------+
|1004|      Anne|Kretchmar|         0.0|2019-07-06 18:30:...|
|1002|    George|   Bailey|       1.875|2019-07-06 18:30:...|
|1003|    Edward|   Walker|         5.3|2019-07-06 18:30:...|
|1001|     Sally|   Thomas|         8.1|2019-07-06 18:30:...|
+----+----------+---------+------------+--------------------+

